All of the theory behind this is coming from this paper https://pubs.acs.org/doi/pdf/10.1021/ed070p568

Goal: calculate equilibrium conditions in a stepwise M + L -> ML1 + ML2 ... + MLn complexation equilibrium using the bisection method

Terminology: 
- $x$ = Free ligand
- $L_{tot}$ = Total ligand
- $c_{0}$ = Total metal
- $f(x) = C_{0}/[M] = K_{0}x^{0} + K_{1}x^{1} + K_{2}x^{2} ... $ Important to note here that $K_{2}$ is what you would normally write as $k_{1}*k_{2}$, small ks being the equilibrium constants here
- $g(x) = \frac{\sum{iK_{i}x^{i}}}{\sum{K_{i}x^{i}}} = \frac{xf'(x)}{f(x)}$   This one is the average number of ligands per complex
- $y(x)$ = total ligand concentration (calculated). $y'(x) \geq 1$. $y(x) = x + c_{0}g(x)$

x is always in the interval $[x_{L}, x_{H}]$,

where $x_H$ is $L_{tot}$ and $x_{L}$ is either $L_{tot} - nC_0$ ($L_{tot}$ minus the calculated number of bound ligands) or 0, whatever is biggest 


The bisection method means 
1. Start with the interval $[x_{L}, x_{H}]$
- Find x in the middle of this interval 
- See if $y(x)$ is higher or lower than $L_{tot}$
- If it's lower, restart with $[x, x_{H}]$, if it's higher restart with $[x_{L}, x]$
- When it's good, $y(x)=L_{tot}$

You can calculate the rest of the concentrations from x (See Equation 1 of the paper)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
def general_bisect(interval,function,ref,*kwargs):
    """General function for the bisection algorithm. First argument should be an interval ([x_min,x_max]), second a function y(x) that requires x as a first argument, plus other arguments (*kwargs), third the value against which to compare y. The function will find the mid point in the interval, calculate y(x), compare it with ref, and repeat with a smaller interval until it homes in the right value"""
    x=(interval[0]+interval[1])/2.0
    #print("Trying x = " + str(x))
        
    y=function(x,*kwargs)
    #print("y(x) = " + str(y))

    #recursive until it's done
    if round(y,2) == round(ref,2):   #rounding off because otherwise it took forever
        #print("Done! x = " + str(x))
        return x
    elif y < ref:
        interval[0]=x
        #print("Trying again, interval = "+ str(interval))
        return(general_bisect(interval,function,ref,*kwargs))
    elif y > ref:
        interval[1]=x
        #print("Trying again, interval = "+ str(interval))
        return(general_bisect(interval,function,ref,*kwargs))

In [3]:
def input_number(prompt):
    """Asks user for a number, doesn't stop until it receives a numeric value that it then converts to a float"""
    user_input = input(prompt)
    try:
        float(user_input)
        return float(user_input)
    except ValueError:
        print("Not a number, try again (use . as a decimal separator)")
        return input_number(prompt)

In [4]:
def g_x(n, B_list,x):
    g_top = 0
    g_bottom = 0
    for i in range(n+1):
        g_top += i*B_list[i]*x**(i)
        g_bottom += B_list[i]*x**(i)
    #print("g(x) = "+str(g_top)+"/"+str(g_bottom))
    return (g_top/g_bottom)

In [5]:
def y_x(x,c_0,n,B_list):
    y = x + c_0*g_x(n, B_list,x)
    #print("y(x) = "+str(y))
    return (y)

This is enough for finding x, now to calculate all concentrations from it

In [6]:
def MLn_calculate_everything_from_x(x,n,c_0,B_list):
    results=[x]
    
    #Calculating everything using the equation (1) of the paper. First, f(x)
    f_x = 0
    for i in range(n+1):
        f_x += B_list[i]*x**i
    
    #Second, calculate [MLi] for each i including 0 ([M]), and append each of them to the results list
    for i in range(n+1):
        MLi = (c_0*B_list[i]*x**i)/f_x
        results.append(MLi)
    
    return(results)
    

All functions combined

In [7]:
def MLn_calculate_everything(user_input=True,L_tot=10,c_0=10,n=2,logB_list=[]):
    """Wrapper of previous functions, with user input or input as arguments. If user_input=False, the program takes from the arguments the values of L_tot (Total concentration of ligand, mM), c_0 (Total concentration of metal, mM), n (number of binding equilibria), and B_list, a list of equilibrium constants [B1,B2...Bn] in units of mM-1,mM-2...mM-n.
    
    Returns a list of equilibrium concentrations (in mM) in this order: [Ligand, Metal, ML1, ML2 .... MLn]"""
    
    if(user_input):
        L_tot = input_number("Total concentration of ligand (mM): ")
        c_0 = input_number("Total concentration of metal (mM): ")
        n = int(input_number("Number of possible binding equilibria: "))
        logB_list = [input_number("log B_"+str(i+1)+" (mM-"+str(i+1)+"): ") for i in range(n)]

    B=1
    B_list = [B]  #This includes what the paper calls K0, B0 in this case =1. It's necessary for the way in which we define g
    for logB in logB_list:
        B = 10**(logB)
        B_list.append(B)

    #Calculate initial interval
    x_l=L_tot-n*c_0
    if x_l<0:
        x_l=0
    x_h=L_tot

    x=general_bisect([x_l,x_h],y_x,L_tot,c_0,n,B_list)

    return(MLn_calculate_everything_from_x(x,n,c_0,B_list))
    

In [8]:
def MLn_plot_metal(user_input=True,L_tot=10,c_0_min=0,c_0_max=20,n=2,logB_list=[],n_points=100,plot_metal=False):
    if(user_input):
        L_tot = input_number("Total concentration of ligand (mM): ")
        c_0_min = input_number("Lowest concentration of metal (mM): ")
        c_0_max = input_number("Highest concentration of metal (mM): ")
        n_points = int(input_number("Number of points to calculate: "))
        n = int(input_number("Number of possible binding equilibria: "))
        logB_list = [input_number("log B_"+str(i+1)+" (mM-"+str(i+1)+"): ") for i in range(n)]

    x = [c_0_min+i*(c_0_max-c_0_min)/(n_points-1) for i in range(n_points)]
    results = np.empty((0,n+2))
    for i in x:
        results = np.append(results,np.array([MLn_calculate_everything(user_input=False,L_tot=L_tot,c_0=i,n=n,logB_list=logB_list)]),axis=0)
    plot = plt.figure()
    plt.plot(x,results[:,0],label="[L]")
    if plot_metal:
        plt.plot(x,results[:,1],label="[M]")
    for j in range(2,n+2):
        label="[ML"+str(j-1)+"]"
        plt.plot(x,results[:,j],label=label)
    plt.legend()
    plt.xlabel("Total concentration of metal (mM)")
    plt.ylabel("Concentration of each species (mM)")
    plt.show()

Interactive widget for terpyridines: considers a mono and a bis complex. The x axis of the graph goes to 1.5 the total concentration of terpyridine

In [11]:
@interact(total_tPy=widgets.IntSlider(min=0,max=100,step=5,value=10),logK1=widgets.FloatSlider(min=-1,max=30,step=0.5,value=5),logB2=widgets.FloatSlider(min=-1,max=30,step=0.5,value=10))
def basic_plot_individual_betas(total_tPy,logK1,logB2):
    return (MLn_plot_metal(user_input=False,L_tot=total_tPy,c_0_min=0,n_points=25,c_0_max=total_tPy*1.5,logB_list=[logK1,logB2]))

interactive(children=(IntSlider(value=10, description='total_tPy', step=5), FloatSlider(value=5.0, description…

Run this one if you want the equilibrium concentrations of a specific system:

In [ ]:
MLn_calculate_everything(user_input=True)